# Spin Glass NADE

In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import numpy as np
from numba import jit

## Colab Instructions
Follow these steps if running on Google colaboratory notebook

## Brief Introduction
We aim to construct a neural network (Neural Autoregressive Distribution Estimator) that can efficiently approximate the Boltzmann distribution of an EA spin glass system in equilibrium via the training data obtained using annealed MCMC simulations. See here for more details: https://arxiv.org/abs/2002.04292

The nitty gritty details of the neural network are coded in the library.py whose classes we'll be importing into this notebook directly:

In [2]:
from library import NADE_orig

In [3]:
model = NADE_orig (inshape=(20,20),num_hidden=20)
model.compile (optimizer='SGD')

We have performed MCMC simulations of the EA lattice at T=0.5 externally and loading the numpy data for the same into here. Overall, we have 20000 latices of 20x20 EA lattices with Gaussian couplings to train our network with. In order to reduce the memory burden on the neural network, we made sure that *all latices have spins at (0,0) position = 1* and noting that multiplying the entire lattice by -1 gives an energetically equivalent configuration that the neural network need not learn

In [27]:
t_lattice = np.load ('TrainingData20k.npy')
np.all (t_lattice[:,0,0] == 1) #Checks if (0,0) spins=1

True

In [5]:
train_data = tf.data.Dataset.from_tensor_slices (t_lattice)
train_data = train_data.batch(20)

Though we can use fit() method available for keras, we'll quickly write a custom training loop to keep things transparent and more flexible:

In [8]:
epochs = 1
for epoch in range (epochs):
    for step, data in enumerate(train_data):
        metrics = model.train_step (data)
        if step % 50 == 49:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step+1, float(metrics["loss"]))
            )
            print("%d samples seen so far" % ((step + 1) * 20))

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Traced
Traced
